In [ ]:
#install package
!pip install Sastrawi nltk tensorflow pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
#library
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


In [ ]:
#nltknlpp
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Tahap Import Dataset



In [ ]:
#import dataset
df = pd.read_excel("/content/dataset.xlsx")
df.columns = df.columns.str.replace("\n", "").str.strip()

print("Jumlah Data:", len(df))
display(df.head())


Jumlah Data: 116


,Keterangan,Motivasi,Karir,Rekomendasi
0,"Prestasi yang mendukung ada banyak, sepeti jua...",90,85,90
1,Prestasi kurang; Gambaran karir hanya sekedar ...,80,60,60
2,Pernah mengikuti kompetisi matematika (LSP) da...,80,70,75
3,Prestasi kurang; Gambaran karir hanya sekedar ...,80,65,65
4,Prestasi berupa sertifikat biologi;,80,65,65


Data Processing

In [ ]:
#inisialisasi Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#load stopwords bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

#fungsi Preprocessing
def preprocess_text(text):
    if isinstance(text, str):
        #tokenisasi
        tokens = word_tokenize(text)

        #casefolding (huruf kecil)
        tokens = [word.lower() for word in tokens]

        #hapus tanda baca dan angka
        tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens if not word.isdigit()]

        #stemming
        tokens = [stemmer.stem(word) for word in tokens]

        #stopwords removal
        tokens = [word for word in tokens if word not in stop_words]

        #gabungkan kembali teks
        return " ".join(tokens)

    return text  #mengembalikan string


In [ ]:
#penerapan preprocessing ke dataset
df_clean = df.dropna()  #......
df_clean["Keterangan"] = df_clean["Keterangan"].apply(preprocess_text)
display(df_clean.head())


<ipython-input-6-4b83bf354f51>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Keterangan"] = df_clean["Keterangan"].apply(preprocess_text)


,Keterangan,Motivasi,Karir,Rekomendasi
0,prestasi dukung sepet juara kelas juara oli...,90,85,90
1,prestasi gambar karir dar software tester se...,80,60,60
2,kompetisi matematika lsp fisika osn tingka...,80,70,75
3,prestasi gambar karir dar software tester,80,65,65
4,prestasi upa sertifikat biologi,80,65,65


**Tokenisasi Dan Padding**

In [ ]:
#mengambil teks & label
texts = df_clean["Keterangan"].tolist()
labels = df_clean[['Motivasi', 'Karir', 'Rekomendasi']].values

#inisialisasi tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

#konversi teks menjadi angka
sequences = tokenizer.texts_to_sequences(texts)

#set maksimal panjang teks dalam dataset
max_length = max(len(s) for s in sequences)
max_length = min(max_length, 50)  # Batasi maksimal 50 kata

#padding sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")


**Training Model**

In [ ]:
#model LSTM
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(3, activation='linear')  #sigmoid
])

#run modwl
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

labels = labels * 100

#train model
model.fit(padded_sequences, labels, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 395ms/step - loss: 53055920.0000 - mae: 7123.2280 - val_loss: 55836940.0000 - val_mae: 7375.6807
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 53751132.0000 - mae: 7176.7168 - val_loss: 55828240.0000 - val_mae: 7375.0894
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - loss: 54474300.0000 - mae: 7238.7417 - val_loss: 55818828.0000 - val_mae: 7374.4512
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - loss: 52807592.0000 - mae: 7102.3076 - val_loss: 55811208.0000 - val_mae: 7373.9385
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - loss: 54191584.0000 - mae: 7213.7524 - val_loss: 55802416.0000 - val_mae: 7373.3457
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step - loss: 54226944.0000 - mae: 7217.4521 - val_loss: 55793472.0000 - val_mae: 7372.7432
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - loss: 54222424.0000 - mae: 7206.9517 - val_loss: 55784336.0000 - val_mae: 7372.1284
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - loss: 5

**Simpan dan load model**

In [ ]:
model.save("model_lstm.h5")
model = load_model("model_lstm.h5", custom_objects={'mse': "mae"})


In [ ]:
def predict_sentiment(text):
    text = preprocess_text(text)  #prreprocessing text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding="post", truncating="post")

    prediction = model.predict(padded_sequence)

    #ouput prediksi
    print(f"Prediksi: Motivasi={prediction[0][0]:.2f}, Karier={prediction[0][1]:.2f}, Rekomendasi={prediction[0][2]:.2f}")

# test
text_input = input("Masukkan Teks Wawancara: ")
predict_sentiment(text_input)


Masukkan Teks Wawancara: Sejak kecil saya sangat menyukai teknologi dan bagaimana suatu sistem dapat bekerja secara otomatis. Saya sering membaca buku tentang kecerdasan buatan dan mengikuti berbagai seminar terkait. Saya merasa program studi ini sangat relevan dengan minat saya, terutama dalam bidang machine learning. Selain itu, saya ingin berkontribusi dalam pengembangan teknologi di Indonesia, sehingga saya yakin bahwa dengan menempuh studi di program ini, saya bisa mendapatkan ilmu dan pengalaman yang sesuai untuk mencapai tujuan tersebut
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step
Prediksi: Motivasi=14.22, Karier=12.38, Rekomendasi=9.21


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Push On Github**

In [ ]:
!git clone https://github.com/seccret404/LMM-model-with-LSTM.git

Cloning into 'LMM-model-with-LSTM'...


In [ ]:
%cd LMM-model-with-LSTM/


/content/LMM-model-with-LSTM


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/TA
!find /content/drive/MyDrive -name "*.ipynb"


!mv "/content/drive/MyDrive/TA (1)/TA.ipynb" /content/LMM-model-with-LSTM/


'Matricxcxcxc TA.gdoc'
/content/drive/MyDrive/Folder 1/Colab Notebooks/11421001_Edward Tua Panjaitan_Tugas 1".ipynb
/content/drive/MyDrive/Folder 1/Colab Notebooks/Untitled0.ipynb
/content/drive/MyDrive/Folder 1/Colab Notebooks/11421001_Edward Tua Panjaitan.ipynb
/content/drive/MyDrive/Folder 1/Colab Notebooks/Untitled1.ipynb
/content/drive/MyDrive/Folder 1/Colab Notebooks/proyekPm.ipynb
/content/drive/MyDrive/TA (1)/TA.ipynb


In [49]:
# !git reset --soft HEAD~1
# !git rm --cached TA.ipynb
# !git commit -m "Remove sensitive data from TA.ipynb"
!git push origin main --force

# !git add TA.ipynb
# !git commit -m "Menambahkan notebook TA.ipynb"
# !git push origin main --force



Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 9.77 KiB | 9.77 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token ——————————————————————
remote:        locations:
remote:          - commit: 48c0ea2f7f2049d92447bd03b192a546e72ebb0e
r